In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'd:\\codes\\mlops\\Kidney-Disease-Classification'

In [32]:

import os
import json 
with open('env.json', 'r') as f:
    environ = json.load(f)

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path : Path
    training_path : Path
    params_image_size : list
    all_params : dict
    mlflow_uri : str
    params_batch_size : int
    

In [34]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml , create_direcories 

In [35]:
class ConfigurationManager:
    def __init__(self, CONFIG_FILE_PATH = CONFIG_FILE_PATH, PARAMS_FILE_PATH = PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        create_direcories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path = Path(self.config.training.trained_model_path),
            training_path = Path("artifacts/data_ingestion/kidney-ct-scan-image"),
            params_image_size = self.params.IMAGE_SIZE,
            all_params = self.params,
            mlflow_uri= environ.get("MLFLOW_TRACKING_URI"),
            params_batch_size = self.params.BATCH_SIZE,
        )
        return eval_config

In [36]:
import tensorflow as tf
from src.cnnClassifier.utils.utils import save_json
import mlflow 
import mlflow.keras
from urllib.parse import urlparse

In [37]:
class Evaluation:
    def __init__(self, config : EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        data_generator = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        
        data_flow = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        valid_datagenerator  = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator # type: ignore
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_path,
            subset = "validation",
            shuffle = True,
            **data_flow # type: ignore
        )
        
    @staticmethod
    def load_model(path : Path)->tf.keras.Model:
            return tf.keras.models.load_model(path) #type: ignore
        
    def evaluation(self):
            self.model = self.load_model(self.config.model_path)
            self._valid_generator()
            self.score = self.model.evaluate(self.valid_generator)
            
    def save_score(self):
            scores  = {"loss ": self.score[0], "accuracy" : self.score[1]}
            save_json(path = Path("scores.json"), data= scores)
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="InceptionV3")
            else:
                mlflow.keras.log_model(self.model, "model")

In [38]:
import sys
from src.cnnClassifier.exception.exception  import  customexception

In [39]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(evaluation_config)
    evaluation._valid_generator()
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise customexception(e, sys)

[2025-01-03 18:26:51,827: INFO: utils: yaml file : config\config.yaml read successfully]
[2025-01-03 18:26:51,851: INFO: utils: yaml file : params.yaml read successfully]


[2025-01-03 18:26:51,857: INFO: utils: directory : artifacts created successfully]
Found 93 images belonging to 2 classes.
Found 93 images belonging to 2 classes.
10/10 [==============================] - 11s 641ms/step - loss: 0.4701 - accuracy: 0.8172
[2025-01-03 18:27:10,965: INFO: utils: json file saved at: scores.json]


2025/01/03 18:27:12 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-01-03 18:27:45,914: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 95). These functions will not be directly callable after loading.]
[2025-01-03 18:27:52,765: INFO: builder_impl: Assets written to: C:\Users\Suyash\AppData\Local\Temp\tmp15pieu11\model\data\model\assets]


Registered model 'InceptionV3' already exists. Creating a new version of this model...
2025/01/03 18:29:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: InceptionV3, version 2
Created version '2' of model 'InceptionV3'.
